In [1]:
# !pip install -U datasets
# !pip install -U transformers

In [2]:
# pip install nltk rouge-score bert-score

In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline
)

import torch

from datasets import load_dataset

import gc

import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize

from rouge_score import rouge_scorer

from bert_score import score

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/pragyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
#Importing the dataset
dataset_name = "tatsu-lab/alpaca"
dataset = load_dataset(dataset_name, split="train[-20:]")

In [16]:
model_save_path_llama2 = "/home/pragyan/Desktop/FineTunedModels/FromSaveLlama2"
model_save_path_phi2 = "/home/pragyan/Desktop/FineTunedModels/FromSavePhi2"
model_save_path_mistral = "/home/pragyan/Desktop/FineTunedModels/FromSaveMistral"

In [17]:
# prompt1 = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: What is the capital of Nepal?"
# pipe1 = pipeline(task="text-generation",
#                 model=model,
#                 tokenizer=tokenizer, 
#                 device=device)
# result1 = pipe1(prompt1, max_new_tokens=128)
# print(result1[0]['generated_text'])

In [98]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Load the tokenizer and model
tokenizer_llama2 = AutoTokenizer.from_pretrained(model_save_path_llama2)
model_llama2 = AutoModelForCausalLM.from_pretrained(model_save_path_llama2).to(device)

totalBlueScore_llama2 = []
totalROUEScore_llama2 = []
totalBERTScore_llama2 = []


for dataRow in dataset:
    template = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: "
    if dataRow['input'] == "":
        promptToUse = template + dataRow['instruction']
    else:
        promptToUse = template + dataRow['instruction'] + " ### Input: " + dataRow['input']
    
    pipe1 = pipeline(task="text-generation", model=model_llama2, tokenizer=tokenizer_llama2, device=device, temperature=0.95)
    result1 = pipe1(promptToUse, max_new_tokens=128)
    
    reference = dataRow['output']
    candidate = result1[0]['generated_text']
    print("---------------------------")
    print(reference)
    print("********")
    print(candidate)
    print("---------------------------")
    
    reference_tokens = word_tokenize(dataRow['output'])
    candidate_tokens = word_tokenize(result1[0]['generated_text'])
    
    # print("Score: ")
    # Calculating BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens)
    # print(f"BLEU score: {bleu_score}")
    totalBlueScore_llama2.append(bleu_score)
    
    # Creating a ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    # Calculating ROUGE-L
    scores = scorer.score(reference, candidate)
    # print(f"ROUGE-L score: {scores['rougeL'].fmeasure}")
    totalROUEScore_llama2.append(scores['rougeL'].fmeasure)
    
    # Calculating BERTScore
    P, R, F1 = score([candidate], [reference], lang="en", verbose=True)

    # Printing the F1 score
    # print(f"BERTScore F1: {F1.mean().item()}")
    totalBERTScore_llama2.append(F1.mean().item())
    
    # print("-------")

Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

---------------------------
The alarm clock blared loudly at seven o'clock.
********
Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: Edit the sentence "The alarm clock was ringing loudly at seven o' clock" to make it more concise.

### Response: The alarm clock blared at seven o'clock. 







































































































---------------------------


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.04 seconds, 27.30 sentences/sec


In [99]:
del tokenizer_llama2
del model_llama2
del pipe1

gc.collect()

86

In [100]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Load the tokenizer and model
tokenizer_phi2 = AutoTokenizer.from_pretrained(model_save_path_phi2)
model_phi2 = AutoModelForCausalLM.from_pretrained(model_save_path_phi2).to(device)


totalBlueScore_phi2 = []
totalROUEScore_phi2 = []
totalBERTScore_phi2 = []


for dataRow in dataset:
    template = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: "
    if dataRow['input'] == "":
        promptToUse = template + dataRow['instruction']
    else:
        promptToUse = template + dataRow['instruction'] + " ### Input: " + dataRow['input']
    
    pipe2 = pipeline(task="text-generation", model=model_phi2, tokenizer=tokenizer_phi2, device=device, temperature=0.95)
    result1 = pipe2(promptToUse, max_new_tokens=128)
    
    reference = dataRow['output']
    candidate = result1[0]['generated_text']
    
    print("---------------------------")
    print(reference)
    print("********")
    print(candidate)
    print("---------------------------")
    
    reference_tokens = word_tokenize(dataRow['output'])
    candidate_tokens = word_tokenize(result1[0]['generated_text'])
    
    # print("Score: ")
    # Calculating BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens)
    # print(f"BLEU score: {bleu_score}")
    totalBlueScore_phi2.append(bleu_score)
    
    # Creating a ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    # Calculating ROUGE-L
    scores = scorer.score(reference, candidate)
    # print(f"ROUGE-L score: {scores['rougeL'].fmeasure}")
    totalROUEScore_phi2.append(scores['rougeL'].fmeasure)
    
    # Calculating BERTScore
    P, R, F1 = score([candidate], [reference], lang="en", verbose=True)

    # Printing the F1 score
    # print(f"BERTScore F1: {F1.mean().item()}")
    totalBERTScore_phi2.append(F1.mean().item())
    
    # print("-------")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

---------------------------
The alarm clock blared loudly at seven o'clock.
********
Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: Edit the sentence "The alarm clock was ringing loudly at seven o' clock"

### Response: The alarm clock was blaring loudly at seven o'clock.

The alarm clock was shrieking loudly at seven o'clock.

The alarm clock was shrieking loudly at seven o'clock.

The alarm clock was blaring loudly at seven o'clock.

The alarm clock was blaring loudly at seven o'clock.

The alarm clock was blaring loudly at seven o'clock.

The alarm clock was blaring loudly at seven o'clock.

The alarm clock was blaring loudly at seven o'clock.

The alarm clock
---------------------------


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.04 seconds, 27.32 sentences/sec


In [101]:
del tokenizer_phi2
del model_phi2
del pipe2

gc.collect()

86

In [102]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Load the tokenizer and model
tokenizer_mistral = AutoTokenizer.from_pretrained(model_save_path_mistral)
model_mistral = AutoModelForCausalLM.from_pretrained(model_save_path_mistral).to(device)


totalBlueScore_mistral = []
totalROUEScore_mistral = []
totalBERTScore_mistral = []


for dataRow in dataset:
    template = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: "
    if dataRow['input'] == "":
        promptToUse = template + dataRow['instruction']
    else:
        promptToUse = template + dataRow['instruction'] + " ### Input: " + dataRow['input']
    
    pipe3 = pipeline(task="text-generation", model=model_mistral, tokenizer=tokenizer_mistral, device=device, temperature=0.95)
    result1 = pipe3(promptToUse, max_new_tokens=128)
    
    reference = dataRow['output']
    candidate = result1[0]['generated_text']
    
    print("---------------------------")
    print(reference)
    print("********")
    print(candidate)
    print("---------------------------")
    
    reference_tokens = word_tokenize(dataRow['output'])
    candidate_tokens = word_tokenize(result1[0]['generated_text'])
    
    # print("Score: ")
    # Calculating BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens)
    # print(f"BLEU score: {bleu_score}")
    totalBlueScore_mistral.append(bleu_score)
    
    # Creating a ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

    # Calculating ROUGE-L
    scores = scorer.score(reference, candidate)
    # print(f"ROUGE-L score: {scores['rougeL'].fmeasure}")
    totalROUEScore_mistral.append(scores['rougeL'].fmeasure)
    
    # Calculating BERTScore
    P, R, F1 = score([candidate], [reference], lang="en", verbose=True)

    # Printing the F1 score
    # print(f"BERTScore F1: {F1.mean().item()}")
    totalBERTScore_mistral.append(F1.mean().item())
    
    # print("-------")

Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

---------------------------
The alarm clock blared loudly at seven o'clock.
********
Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: Edit the sentence "The alarm clock was ringing loudly at seven o' clock" to make it more concise. ### Response: The alarm rang at seven o' clock.

### Instruction: Generate a sentence that describes a person who is always late.
### Response: He is always late, never on time.

### Instruction: Generate a sentence that describes a person who is always early.
### Response: She is always early, never late.

### Instruction: Generate a sentence that describes a person who is always on time.
### Response: He is always on time, never late.

### Instruction: Generate a sentence that describes a person who is always
---------------------------


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.04 seconds, 27.18 sentences/sec


In [103]:
print(len(totalBlueScore_llama2))
print(len(totalROUEScore_llama2))
print(len(totalBERTScore_llama2))

print(len(totalBlueScore_phi2))
print(len(totalROUEScore_phi2))
print(len(totalBERTScore_phi2))

print(len(totalBlueScore_mistral))
print(len(totalROUEScore_mistral))
print(len(totalBERTScore_mistral))


20
20
20
20
20
20
20
20
20


In [104]:
print("Final Scores with temperature value to 0.95: ")

print("------------  Llama 2 -----------------------------")
meanBlueLlama2 = sum(totalBlueScore_llama2) / len(totalBlueScore_llama2)
meanROUEScoreLlama2 = sum(totalROUEScore_llama2) / len(totalROUEScore_llama2)
meanBERTScoreLlama2 = sum(totalBERTScore_llama2) / len(totalBERTScore_llama2)
print("Blue: ", meanBlueLlama2)
print("ROUGE: ", meanROUEScoreLlama2)
print("BERTScore: ", meanBERTScoreLlama2)


print("------------  Phi 2 -----------------------------")
meanBluePhi2 = sum(totalBlueScore_phi2) / len(totalBlueScore_phi2)
meanROUEScorePhi2 = sum(totalROUEScore_phi2) / len(totalROUEScore_phi2)
meanBERTScorePhi2 = sum(totalBERTScore_phi2) / len(totalBERTScore_phi2)
print("Blue: ", meanBluePhi2)
print("ROUGE: ", meanROUEScorePhi2)
print("BERTScore: ", meanBERTScorePhi2)


print("------------  MISTRAL -----------------------------")
meanBlueMistral = sum(totalBlueScore_mistral) / len(totalBlueScore_mistral)
meanROUEScoreMistral = sum(totalROUEScore_mistral) / len(totalROUEScore_mistral)
meanBERTScoreMistral = sum(totalBERTScore_mistral) / len(totalBERTScore_mistral)
print("Blue: ", meanBlueMistral)
print("ROUGE: ", meanROUEScoreMistral)
print("BERTScore: ", meanBERTScoreMistral)

Final Scores with temperature value to 0.95: 
------------  Llama 2 -----------------------------
Blue:  0.06750837098152963
ROUGE:  0.21668371252206717
BERTScore:  0.8427776753902435
------------  Phi 2 -----------------------------
Blue:  0.07853504038603079
ROUGE:  0.2314906368443148
BERTScore:  0.855524867773056
------------  MISTRAL -----------------------------
Blue:  0.06801136698769118
ROUGE:  0.20915747976241508
BERTScore:  0.8460512310266495


In [105]:
del tokenizer_mistral
del model_mistral
del pipe3

gc.collect()

86